In [ ]:
%matplotlib widget

In [ ]:
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal

from tqdm import tqdm
from ipywidgets import interact

In [3]:
def sobel_filter_x():
    return np.array([[-1,0,1],[-2,0,2],[-1,0,1]])

def sobel_filter_y():
    return np.array([[1,2,1], [0,0,0], [-1,-2,-1]])

def scharr_filter_combined():
    """https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html"""
    return np.array([[ -3-3j, 0-10j,  +3 -3j],
                     [-10+0j, 0+ 0j, +10 +0j],
                     [ -3+3j, 0+10j,  +3 +3j]])

In [4]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(sobel_filter_x(), cmap='gray')
axes[1].imshow(sobel_filter_y(), cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
def load_image(image_file):
    image = Image.open(image_file)
    image_array = np.array(image.getdata()).reshape(image.height, image.width, -1)
    return image_array

In [6]:
image = load_image('persistence-of-memory-small.jpg')

In [7]:
fig, ax = plt.subplots(1, 1)
ax.imshow(image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
def brightness(image):
    return image.sum(axis=-1) / 3

def brightness_perceived(image):
    return np.sqrt(0.299*(image[...,0]**2) + 0.587*(image[..., 1]**2) + 0.114*(image[..., 2]**2))

In [9]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(brightness(image), cmap='gray')
axes[1].imshow(brightness_perceived(image), cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
image_grey_scale = brightness(image)

In [26]:
def compute_energies(image_grey_scale):
    gx = scipy.signal.convolve2d(image_grey_scale, sobel_filter_x(), mode='same')
    gy = scipy.signal.convolve2d(image_grey_scale, sobel_filter_y(), mode='same')
    g = np.sqrt(gx**2 + gy**2)
    return g

In [27]:
g = compute_energies(image_grey_scale)

In [28]:
fig, ax = plt.subplots(1, 1)
ax.imshow(g, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def find_path_naive(gradient):
    pass

In [78]:
def find_path_greedy(energies):
    rows, cols = energies.shape[0], energies.shape[1]
    col = np.argmin(energies[0])
    path = [col]
    for row in range(1, rows):
        col_relative = np.argmin(energies[row, max(0, col-1):min(col+2, cols)])
        col = min(cols, max(0, col_relative - 1 + col))
        path.append(col)
    return np.array(path)

In [79]:
def compute_accumulated_energies(energies):
    accumulated_energies = np.zeros(energies.shape)
    accumulated_energies[-1, ...] = energies[-1, ...]
    rows, cols = energies.shape[0], energies.shape[1]
    rows_reversed = reversed(range(1, rows))
    for row in rows_reversed:
        for col in range(cols):
            accumulated_energies[row - 1, col] = accumulated_energies[row, max(0, col-1):min(col+2, cols)].min()
        accumulated_energies[row - 1] += energies[row - 1]
    return accumulated_energies

def find_path_dynamic(energies=None, accumulated_energies=None):
    """Find global least-energy seam by pre-computing accumulated "energies" of image gradient from top to bottom""" 
    if accumulated_energies is None:
        accumulated_energies = compute_accumulated_energies(energies)
    path = find_path_greedy(accumulated_energies)
    return path

In [80]:
path_greedy = find_path_greedy(g)

In [177]:
acc_energies = compute_accumulated_energies(g)
path_dynamic = find_path_dynamic(None, acc_energies)

In [180]:
def highlight_pixels(image, indices):
    rows = np.array(range(image.shape[0]))
    n_cols = image.shape[1]

    if image.ndim == 3:
        replace = (255, 0, 255)
    else:
        replace = 0

    image[rows, indices, ...] = replace
    right_neighbour = np.clip(indices+1, a_min=None, a_max=n_cols)
    image[rows, right_neighbour, ...] = replace
    left_neighbour = np.clip(indices-1, a_min=0, a_max=None)
    image[rows, left_neighbour, ...] = replace
    return image

def visualize_path(ax, image, path, imshow_obj=None):
    image_to_show = np.array(image)
    image_to_show = highlight_pixels(image_to_show, path)
    if imshow_obj:
        imshow_obj.set_data(image_to_show)
    else:
        ax.imshow(image_to_show)

In [181]:
fig, axes = plt.subplots(2, 2)
visualize_path(axes[0][0], image, path_greedy)
visualize_path(axes[0][1], image, path_dynamic)
visualize_path(axes[1][0], acc_energies, path_dynamic)

normalized_accumulated_energies = accumulated_energies / accumulated_energies.max()  * 255
mixed = image + normalized_accumulated_energies[:, :, np.newaxis]
mixed /= mixed.max()
visualize_path(axes[1][1], mixed, path_dynamic)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [182]:
def construct_cropped_images(image, n_cols, reaccumulate_energies_every=10):
    images = []
    paths = []
    accumulated_energies = []

    rows = range(image.shape[0])
    
    _image = np.array(image)

    for i in tqdm(range(n_cols)):
        if i % reaccumulate_energies_every == 0:
            grey_scale = brightness(_image)
            g = compute_energies(grey_scale)
            e = compute_accumulated_energies(g)
        else:
            e = e[~bool_map[..., 0]].reshape(e.shape[0], e.shape[1] - 1)

        path = find_path_dynamic(None, e)
        
        bool_map = np.zeros(_image.shape, dtype=np.bool)
        bool_map[rows, path] = True

        _image = _image[~bool_map].reshape(_image.shape[0], _image.shape[1] - 1, -1)

        images.append(_image)
        paths.append(path)
        accumulated_energies.append(e)
    return images, paths, accumulated_energies

In [183]:
images, paths, accumulated_energies = construct_cropped_images(image, 250, reaccumulate_energies_every=10)

100%|██████████| 250/250 [00:10<00:00, 24.21it/s]


In [184]:
fig, ax = plt.subplots(1, 1)
imshow_obj = ax.imshow(images[0])

def update(index=0):
    visualize_path(None, images[index], paths[index], imshow_obj=imshow_obj)

interact(update, index=(0, len(images) - 1));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='index', max=249), Output()), _dom_classes=('widget-inter…

In [185]:
fig, ax = plt.subplots(1, 2)
imshow_obj1 = ax[0].imshow(images[0])
imshow_obj2 = ax[1].imshow(accumulated_energies[0])

def update(index=0):
    visualize_path(None, images[index], paths[index], imshow_obj=imshow_obj1)
    visualize_path(None, accumulated_energies[index], paths[index], imshow_obj=imshow_obj2)

interact(update, index=(0, len(images) - 1));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='index', max=249), Output()), _dom_classes=('widget-inter…